In [46]:
!pip install pycryptodome
!pip install numpy

# TLS with random block size and random padding

In [8]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

import struct

# Define constants
AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding

# Function to add random padding to data
def add_random_padding(data):
    padding_length = get_random_bytes(1)[0] % RANDOM_PADDING_MAX
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

# Encrypt with AES-GCM, adding random padding to the plaintext
def encrypt_with_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    # Add random padding to plaintext
    padded_data, padding_length = add_random_padding(plaintext)
    
    # Encrypt padded data
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    
    # Structure the output as nonce + padding length + ciphertext + tag
    record = nonce + struct.pack("B", padding_length) + ciphertext + tag
    return record

# Decrypt function
def decrypt_with_padding(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    # Remove padding
    plaintext = padded_data[:-padding_length] if padding_length > 0 else padded_data
    return plaintext

# Example usage
key = get_random_bytes(AES_KEY_SIZE)
plaintext = b"It Is very important and secret text"

# Encrypt
encrypted_record = encrypt_with_padding(plaintext, key)

# Decrypt
decrypted_plaintext = decrypt_with_padding(encrypted_record, key)
print("Original:", plaintext)
print("Decrypted:", decrypted_plaintext)
print("Encrypted:", encrypted_record)


Original: b'It Is very important and secret text'
Decrypted: b'It Is very important and secret text'
Encrypted: b'\xc6\x0e\xca\x8e\x17\x7f\xe2\x9b\xba\xb7\x89\x8d\x16\xe2\x16D\x9d\x1d\xd4#\xaf`\xaf<\xfd\xe5o\xa1\xe8t\x13E\xeer\x91\x87\x1cJ"\x8d\xd9\xc6\x0e\xdf\xf7u+t\x9b8\xdd\xd3\xb7\xab\xf4\xe1\x8d\xc4|\x81\xeb\x8d\x02\x98\xb4\xffF\x16\xef\'\xcb\\\xf5\xdb\xd9%\x00\xcb]\x84\xdb=Q<Jy\x07'


## TLS with Diffie-Hellman key exchange(prime number of size 2048 bits)

In [10]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import struct
import os
from sympy import nextprime

# Define constants
AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
SESSION_KEY_SIZE = 32  # Size for session keys

# Function to generate a large prime number
def generate_large_prime(bits=2048):
    # Generate a random number of the specified bit size
    random_num = os.urandom(bits // 8)
    # Convert to integer and find the next prime
    return nextprime(int.from_bytes(random_num, 'big'))

# Function to simulate a Diffie-Hellman key exchange
def key_exchange(private_key, prime_p, generator_g):
    # Generate a public key
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

# Function to derive a shared session key
def derive_session_key(private_key, other_public_key, prime_p):
    # Calculate the shared secret
    shared_secret = pow(other_public_key, private_key, prime_p)
    
    # Derive a session key from the shared secret using a hash function
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

# Function to add random padding to data
def add_random_padding(data):
    padding_length = get_random_bytes(1)[0] % RANDOM_PADDING_MAX
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

# Encrypt with AES-GCM, adding random padding to the plaintext
def encrypt_with_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    # Add random padding to plaintext
    padded_data, padding_length = add_random_padding(plaintext)
    
    # Encrypt padded data
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    
    # Structure the output as nonce + padding length + ciphertext + tag
    record = nonce + struct.pack("B", padding_length) + ciphertext + tag
    return record

# Decrypt function
def decrypt_with_padding(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    # Remove padding
    plaintext = padded_data[:-padding_length] if padding_length > 0 else padded_data
    return plaintext

# Example usage
def main():
    # Generate large prime
    prime_p = generate_large_prime()
    GENERATOR_G = 2  # Typically, a small number like 2 is used as a generator

    # Generate secure random private keys
    alice_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1

    # Alice and Bob exchange public keys
    alice_public = key_exchange(alice_private, prime_p, GENERATOR_G)
    bob_public = key_exchange(bob_private, prime_p, GENERATOR_G)

    # Derive session keys
    alice_session_key = derive_session_key(alice_private, bob_public, prime_p)
    bob_session_key = derive_session_key(bob_private, alice_public, prime_p)

    plaintext = b"It Is very important and secret text"
    
    # Encrypt using Alice's session key
    encrypted_record = encrypt_with_padding(plaintext, alice_session_key)
    
    # Decrypt using Alice's session key (in practice, Bob would decrypt with his session key)
    decrypted_plaintext = decrypt_with_padding(encrypted_record, alice_session_key)
    
    print("Original:", plaintext)
    print("Decrypted:", decrypted_plaintext)
    print("Encrypted:", encrypted_record)

if __name__ == "__main__":
    main()


Original: b'It Is very important and secret text'
Decrypted: b'It Is very important and secret text'
Encrypted: b'\x1e1\x05#4\xf6\xb6\x87MY\x97l\xb0\x90\xa2%l\xce\xfd~\xc5=\x00\x81V\xfc\xba\xd1\xd67E\xd2\x7f\xf3p6E\xd6\xab\x85!j\xc2\xbdbb&\xc5\xec\x1d\xb5\xd6\xe77M\xc9\x15\xfd\x92\xf4\xe3\x07\xb7&c\x8b\xa0\xe7\xaf3\xed{\xa7\x83\xca\xb3!U$d\xf1\x00\xd8|f\xc8\x90)\xfd\xa9\xb4\x0f\x99\xa0d\x7f\x9d\x9c\xea\x7f\xba\xe9\xb5\x90\xf0\xde\x9a\xe8g\xb4\xee\x15\xbf\xb2\xb0\xf0b \x14\xe0K\x14L\xc6\xc2Bj\x12_Z\xc9\xb8\xed~\xfd\xc9\xdd\xa5/w\x9ez\x7f;\xa8I\xf2\x95G\x85-\xca\xc1,.\xc6Tb\xd1=p\x17,\xf0T\x96\xa9\xb2r\xe8/\xd0\x9fRn\xa4\x8dD\x19\x88\x0b\xbc\xfe\xee\xec\xbeH)\xc4\xb8,\xe5\xf6\xd7x\x11\x06\xefy\xe7J>\xb7\x96\r\x8f\xfcuw\x0e\xc3\x1b\x07[-V\x0b\x07\xb7\xc0yX\xa6\xf6\xab\xe1\xf8\x07R\xcc\xa4\x89PX\xecS\x8b\xb1j~P'


## Comparision of Encryption/Decryption time overhead of normal TLS and Random TLS

In [12]:
import time
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import struct
import os
from sympy import nextprime

# Define constants
AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
FIXED_PADDING_SIZE = 16  # Fixed padding size for regular TLS
SESSION_KEY_SIZE = 32  # Size for session keys
NUM_ITERATIONS = 1000  # Number of iterations for performance measurement

# Function to generate a large prime number
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

# Function to simulate a Diffie-Hellman key exchange
def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

# Function to derive a shared session key
def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

# Function to add random padding to data
def add_random_padding(data):
    padding_length = get_random_bytes(1)[0] % RANDOM_PADDING_MAX
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

# Function to add fixed padding to data
def add_fixed_padding(data):
    padding = get_random_bytes(FIXED_PADDING_SIZE)
    return data + padding

# Encrypt with AES-GCM, adding random padding to the plaintext
def encrypt_with_random_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", padding_length) + ciphertext + tag
    return record

# Encrypt with AES-GCM, adding fixed padding to the plaintext
def encrypt_with_fixed_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = add_fixed_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", FIXED_PADDING_SIZE) + ciphertext + tag
    return record

# Decrypt function (same for both)
def decrypt_with_padding(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    if padding_length > 0:
        return padded_data[:-padding_length]
    return padded_data

# Example usage
def main():
    # Generate large prime
    prime_p = generate_large_prime()
    GENERATOR_G = 2  # Typically, a small number like 2 is used as a generator

    # Generate secure random private keys
    alice_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1

    # Alice and Bob exchange public keys
    alice_public = key_exchange(alice_private, prime_p, GENERATOR_G)
    bob_public = key_exchange(bob_private, prime_p, GENERATOR_G)

    # Derive session keys
    alice_session_key = derive_session_key(alice_private, bob_public, prime_p)
    bob_session_key = derive_session_key(bob_private, alice_public, prime_p)

    plaintext = b"It Is very important and secret text" * 1000  # Increase size for performance testing
    
    # Measure performance of random padding
    random_time = 0
    for _ in range(NUM_ITERATIONS):
        start_time = time.perf_counter()
        encrypted_record_random = encrypt_with_random_padding(plaintext, alice_session_key)
        decrypted_plaintext_random = decrypt_with_padding(encrypted_record_random, alice_session_key)
        random_time += (time.perf_counter() - start_time)
    
    # Measure performance of fixed padding
    fixed_time = 0
    for _ in range(NUM_ITERATIONS):
        start_time = time.perf_counter()
        encrypted_record_fixed = encrypt_with_fixed_padding(plaintext, alice_session_key)
        decrypted_plaintext_fixed = decrypt_with_padding(encrypted_record_fixed, alice_session_key)
        fixed_time += (time.perf_counter() - start_time)
    
    # Print results
    print("Original:", plaintext[:50], "...")  # Print a portion for brevity
    print("Decrypted (Random Padding):", decrypted_plaintext_random[:50], "...")
    print("Decrypted (Fixed Padding):", decrypted_plaintext_fixed[:50], "...")
    print(f"Average Random Padding Encryption/Decryption Time: {random_time / NUM_ITERATIONS:.6f} seconds")
    print(f"Average Fixed Padding Encryption/Decryption Time: {fixed_time / NUM_ITERATIONS:.6f} seconds")

if __name__ == "__main__":
    main()


Original: b'It Is very important and secret textIt Is very imp' ...
Decrypted (Random Padding): b'It Is very important and secret textIt Is very imp' ...
Decrypted (Fixed Padding): b'It Is very important and secret textIt Is very imp' ...
Average Random Padding Encryption/Decryption Time: 0.000686 seconds
Average Fixed Padding Encryption/Decryption Time: 0.000644 seconds


## Time Analysis of 100 random strings of length 50 with regular TLS and Random TLS

In [43]:
import random
import string

def generate_random_strings(num_strings=100, string_length=50):
    with open('random_strings.txt', 'w') as f:
        for i in range(num_strings):
            random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=string_length))
            f.write(random_string + '\n')

if __name__ == "__main__":
    generate_random_strings()


In [13]:
import time
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import struct
import os
from sympy import nextprime

# Define constants
AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
FIXED_PADDING_SIZE = 16  # Fixed padding size for regular TLS
SESSION_KEY_SIZE = 32  # Size for session keys
NUM_ITERATIONS = 10  # Number of iterations for performance measurement

# Function to generate a large prime number
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

# Function to simulate a Diffie-Hellman key exchange
def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

# Function to derive a shared session key
def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

# Function to add random padding to data
def add_random_padding(data):
    padding_length = get_random_bytes(1)[0] % RANDOM_PADDING_MAX
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

# Function to add fixed padding to data
def add_fixed_padding(data):
    padding = get_random_bytes(FIXED_PADDING_SIZE)
    return data + padding

# Encrypt with AES-GCM, adding random padding to the plaintext
def encrypt_with_random_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", padding_length) + ciphertext + tag
    return record

# Encrypt with AES-GCM, adding fixed padding to the plaintext
def encrypt_with_fixed_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = add_fixed_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", FIXED_PADDING_SIZE) + ciphertext + tag
    return record

# Decrypt function (same for both)
def decrypt_with_padding(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    if padding_length > 0:
        return padded_data[:-padding_length]
    return padded_data

# Example usage
def main():
    # Generate large prime
    prime_p = generate_large_prime()
    GENERATOR_G = 2  # Typically, a small number like 2 is used as a generator

    # Generate secure random private keys
    alice_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1

    # Alice and Bob exchange public keys
    alice_public = key_exchange(alice_private, prime_p, GENERATOR_G)
    bob_public = key_exchange(bob_private, prime_p, GENERATOR_G)

    # Derive session keys
    alice_session_key = derive_session_key(alice_private, bob_public, prime_p)
    bob_session_key = derive_session_key(bob_private, alice_public, prime_p)

    # Read strings from the text file
    with open('random_strings.txt', 'r') as f:
        strings = [line.strip().encode() for line in f.readlines()]

    for plaintext in strings:
        # Measure performance of random padding
        random_time = 0
        for _ in range(NUM_ITERATIONS):
            start_time = time.perf_counter()
            encrypted_record_random = encrypt_with_random_padding(plaintext, alice_session_key)
            decrypted_plaintext_random = decrypt_with_padding(encrypted_record_random, alice_session_key)
            random_time += (time.perf_counter() - start_time)
        
        # Measure performance of fixed padding
        fixed_time = 0
        for _ in range(NUM_ITERATIONS):
            start_time = time.perf_counter()
            encrypted_record_fixed = encrypt_with_fixed_padding(plaintext, alice_session_key)
            decrypted_plaintext_fixed = decrypt_with_padding(encrypted_record_fixed, alice_session_key)
            fixed_time += (time.perf_counter() - start_time)
        
        # Print results for each string
        print(f"Original: {plaintext[:50]}...")  # Print a portion for brevity
        print(f"Decrypted (Random Padding): {decrypted_plaintext_random[:50]}...")
        print(f"Decrypted (Fixed Padding): {decrypted_plaintext_fixed[:50]}...")
        print(f"Average Random Padding Time: {random_time / NUM_ITERATIONS:.6f} seconds")
        print(f"Average Fixed Padding Time: {fixed_time / NUM_ITERATIONS:.6f} seconds")
        print("---")

if __name__ == "__main__":
    main()


Original: b'pe3kGloh617WeyTX5L4NxO7CwjiZARvnQrVONA0tMw51MisjXg'...
Decrypted (Random Padding): b'pe3kGloh617WeyTX5L4NxO7CwjiZARvnQrVONA0tMw51MisjXg'...
Decrypted (Fixed Padding): b'pe3kGloh617WeyTX5L4NxO7CwjiZARvnQrVONA0tMw51MisjXg'...
Average Random Padding Time: 0.000556 seconds
Average Fixed Padding Time: 0.000520 seconds
---
Original: b'oO5gcnkoKbCLOEM3FZIE5E5VTgByVa1qrgoobhkEcAz5WBUqBx'...
Decrypted (Random Padding): b'oO5gcnkoKbCLOEM3FZIE5E5VTgByVa1qrgoobhkEcAz5WBUqBx'...
Decrypted (Fixed Padding): b'oO5gcnkoKbCLOEM3FZIE5E5VTgByVa1qrgoobhkEcAz5WBUqBx'...
Average Random Padding Time: 0.000724 seconds
Average Fixed Padding Time: 0.000524 seconds
---
Original: b'Qi3pnn2l9cw1CjgIPBStvoXIOPgvUQDIDk5rGKfRV1OyPndNoM'...
Decrypted (Random Padding): b'Qi3pnn2l9cw1CjgIPBStvoXIOPgvUQDIDk5rGKfRV1OyPndNoM'...
Decrypted (Fixed Padding): b'Qi3pnn2l9cw1CjgIPBStvoXIOPgvUQDIDk5rGKfRV1OyPndNoM'...
Average Random Padding Time: 0.000714 seconds
Average Fixed Padding Time: 0.000515 seconds
---
Origina

Original: b'TK5mciCzHIcKQC4OfbdZuBcIsqKUj8KYXE95E4hNIQvymFrfgj'...
Decrypted (Random Padding): b'TK5mciCzHIcKQC4OfbdZuBcIsqKUj8KYXE95E4hNIQvymFrfgj'...
Decrypted (Fixed Padding): b'TK5mciCzHIcKQC4OfbdZuBcIsqKUj8KYXE95E4hNIQvymFrfgj'...
Average Random Padding Time: 0.001001 seconds
Average Fixed Padding Time: 0.000516 seconds
---
Original: b'RcIyNxCFN8E0r0wOCXPfuq8rtpPBJXIUKsLH1I72eaJR3STqcb'...
Decrypted (Random Padding): b'RcIyNxCFN8E0r0wOCXPfuq8rtpPBJXIUKsLH1I72eaJR3STqcb'...
Decrypted (Fixed Padding): b'RcIyNxCFN8E0r0wOCXPfuq8rtpPBJXIUKsLH1I72eaJR3STqcb'...
Average Random Padding Time: 0.000738 seconds
Average Fixed Padding Time: 0.000513 seconds
---
Original: b'IA1DUTi4y31gxTtqSXbZlCSkE1V69qojn8XejFmfGBqEyYeeWd'...
Decrypted (Random Padding): b'IA1DUTi4y31gxTtqSXbZlCSkE1V69qojn8XejFmfGBqEyYeeWd'...
Decrypted (Fixed Padding): b'IA1DUTi4y31gxTtqSXbZlCSkE1V69qojn8XejFmfGBqEyYeeWd'...
Average Random Padding Time: 0.000555 seconds
Average Fixed Padding Time: 0.000382 seconds
---
Origina

Original: b'1CwK7KImjckFvARG9QOhqi3fIylcd8WHHhPbmL6eQupl68giZM'...
Decrypted (Random Padding): b'1CwK7KImjckFvARG9QOhqi3fIylcd8WHHhPbmL6eQupl68giZM'...
Decrypted (Fixed Padding): b'1CwK7KImjckFvARG9QOhqi3fIylcd8WHHhPbmL6eQupl68giZM'...
Average Random Padding Time: 0.000964 seconds
Average Fixed Padding Time: 0.000461 seconds
---
Original: b'j0NWqtfJP9Hr4SHEABKXBowFTiTVYwDLaPkHut2t5RrFlmHyds'...
Decrypted (Random Padding): b'j0NWqtfJP9Hr4SHEABKXBowFTiTVYwDLaPkHut2t5RrFlmHyds'...
Decrypted (Fixed Padding): b'j0NWqtfJP9Hr4SHEABKXBowFTiTVYwDLaPkHut2t5RrFlmHyds'...
Average Random Padding Time: 0.000734 seconds
Average Fixed Padding Time: 0.000480 seconds
---
Original: b'ZA5nwH7EDrDeg0Uv3FCJfIKZ7X1dP0PX2ffk3gNSe77SsRPgoy'...
Decrypted (Random Padding): b'ZA5nwH7EDrDeg0Uv3FCJfIKZ7X1dP0PX2ffk3gNSe77SsRPgoy'...
Decrypted (Fixed Padding): b'ZA5nwH7EDrDeg0Uv3FCJfIKZ7X1dP0PX2ffk3gNSe77SsRPgoy'...
Average Random Padding Time: 0.000688 seconds
Average Fixed Padding Time: 0.000517 seconds
---
Origina

Original: b'vmLvv8hDbMuE8az5jkGYpJTuKfGVivcC7di8u6DpKRRoEf5fKW'...
Decrypted (Random Padding): b'vmLvv8hDbMuE8az5jkGYpJTuKfGVivcC7di8u6DpKRRoEf5fKW'...
Decrypted (Fixed Padding): b'vmLvv8hDbMuE8az5jkGYpJTuKfGVivcC7di8u6DpKRRoEf5fKW'...
Average Random Padding Time: 0.000941 seconds
Average Fixed Padding Time: 0.000507 seconds
---
Original: b'PWVjlGUCGl0cJEPFe3BDQsMhU9rEs068MsKqabYHFKweUTcp2j'...
Decrypted (Random Padding): b'PWVjlGUCGl0cJEPFe3BDQsMhU9rEs068MsKqabYHFKweUTcp2j'...
Decrypted (Fixed Padding): b'PWVjlGUCGl0cJEPFe3BDQsMhU9rEs068MsKqabYHFKweUTcp2j'...
Average Random Padding Time: 0.000725 seconds
Average Fixed Padding Time: 0.000514 seconds
---


## Statastical analysis of Encryption/Decryption time(performance overhead) of 100 random strings of length 50 over 100 iteration

In [14]:
import time
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import struct
import os
from sympy import nextprime
import numpy as np

# Define constants
AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
FIXED_PADDING_SIZE = 16  # Fixed padding size for regular TLS
SESSION_KEY_SIZE = 32  # Size for session keys
NUM_ITERATIONS = 100  # Number of iterations for performance measurement

# Function to generate a large prime number
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

# Function to simulate a Diffie-Hellman key exchange
def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

# Function to derive a shared session key
def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

# Function to add random padding to data
def add_random_padding(data):
    padding_length = get_random_bytes(1)[0] % RANDOM_PADDING_MAX
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

# Function to add fixed padding to data
def add_fixed_padding(data):
    padding = get_random_bytes(FIXED_PADDING_SIZE)
    return data + padding

# Encrypt with AES-GCM, adding random padding to the plaintext
def encrypt_with_random_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", padding_length) + ciphertext + tag
    return record

# Encrypt with AES-GCM, adding fixed padding to the plaintext
def encrypt_with_fixed_padding(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = add_fixed_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    record = nonce + struct.pack("B", FIXED_PADDING_SIZE) + ciphertext + tag
    return record

# Decrypt function (same for both)
def decrypt_with_padding(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    if padding_length > 0:
        return padded_data[:-padding_length]
    return padded_data

# Example usage
def main():
    # Generate large prime
    prime_p = generate_large_prime()
    GENERATOR_G = 2  # Typically, a small number like 2 is used as a generator

    # Generate secure random private keys
    alice_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1

    # Alice and Bob exchange public keys
    alice_public = key_exchange(alice_private, prime_p, GENERATOR_G)
    bob_public = key_exchange(bob_private, prime_p, GENERATOR_G)

    # Derive session keys
    alice_session_key = derive_session_key(alice_private, bob_public, prime_p)
    bob_session_key = derive_session_key(bob_private, alice_public, prime_p)

    # Read strings from the text file
    with open('random_strings.txt', 'r') as f:
        strings = [line.strip().encode() for line in f.readlines()]

    # Store timing results
    random_times = []
    fixed_times = []

    for plaintext in strings:
        print(f"\nProcessing: {plaintext[:50]}...")  # Print a portion for brevity

        # Measure performance of random padding
        for _ in range(NUM_ITERATIONS):
            start_time = time.perf_counter()
            encrypted_record_random = encrypt_with_random_padding(plaintext, alice_session_key)
            decrypted_plaintext_random = decrypt_with_padding(encrypted_record_random, alice_session_key)
            elapsed_time = time.perf_counter() - start_time
            random_times.append(elapsed_time)

            print(f"Random Padding Time: {elapsed_time:.6f}s")
        
        # Measure performance of fixed padding
        for _ in range(NUM_ITERATIONS):
            start_time = time.perf_counter()
            encrypted_record_fixed = encrypt_with_fixed_padding(plaintext, alice_session_key)
            decrypted_plaintext_fixed = decrypt_with_padding(encrypted_record_fixed, alice_session_key)
            elapsed_time = time.perf_counter() - start_time
            fixed_times.append(elapsed_time)

            print(f"Fixed Padding Time: {elapsed_time:.6f}s")

    # Calculate statistics for random padding
    random_mean = np.mean(random_times)
    random_median = np.median(random_times)
    random_std = np.std(random_times)
    random_min = np.min(random_times)
    random_max = np.max(random_times)

    # Calculate statistics for fixed padding
    fixed_mean = np.mean(fixed_times)
    fixed_median = np.median(fixed_times)
    fixed_std = np.std(fixed_times)
    fixed_min = np.min(fixed_times)
    fixed_max = np.max(fixed_times)

    # Print statistics
    print("\nStatistics:")
    print(f"Random Padding: Mean = {random_mean:.6f}s, Median = {random_median:.6f}s, Std Dev = {random_std:.6f}s, Min = {random_min:.6f}s, Max = {random_max:.6f}s")
    print(f"Fixed Padding: Mean = {fixed_mean:.6f}s, Median = {fixed_median:.6f}s, Std Dev = {fixed_std:.6f}s, Min = {fixed_min:.6f}s, Max = {fixed_max:.6f}s")

if __name__ == "__main__":
    main()



Processing: b'pe3kGloh617WeyTX5L4NxO7CwjiZARvnQrVONA0tMw51MisjXg'...
Random Padding Time: 0.001579s
Random Padding Time: 0.000839s
Random Padding Time: 0.001127s
Random Padding Time: 0.001169s
Random Padding Time: 0.001140s
Random Padding Time: 0.001104s
Random Padding Time: 0.001146s
Random Padding Time: 0.000840s
Random Padding Time: 0.001456s
Random Padding Time: 0.001145s
Random Padding Time: 0.000798s
Random Padding Time: 0.000798s
Random Padding Time: 0.001088s
Random Padding Time: 0.001133s
Random Padding Time: 0.001127s
Random Padding Time: 0.001129s
Random Padding Time: 0.001135s
Random Padding Time: 0.001143s
Random Padding Time: 0.001139s
Random Padding Time: 0.001138s
Random Padding Time: 0.001133s
Random Padding Time: 0.001142s
Random Padding Time: 0.001160s
Random Padding Time: 0.000995s
Random Padding Time: 0.001374s
Random Padding Time: 0.000927s
Random Padding Time: 0.001131s
Random Padding Time: 0.000844s
Random Padding Time: 0.001353s
Random Padding Time: 0.000932s


Fixed Padding Time: 0.001541s
Fixed Padding Time: 0.001007s
Fixed Padding Time: 0.001693s
Fixed Padding Time: 0.001352s
Fixed Padding Time: 0.001252s
Fixed Padding Time: 0.000872s
Fixed Padding Time: 0.001437s
Fixed Padding Time: 0.001166s
Fixed Padding Time: 0.001144s
Fixed Padding Time: 0.001201s
Fixed Padding Time: 0.001010s
Fixed Padding Time: 0.001173s
Fixed Padding Time: 0.001095s
Fixed Padding Time: 0.001159s
Fixed Padding Time: 0.001162s
Fixed Padding Time: 0.001157s
Fixed Padding Time: 0.001149s
Fixed Padding Time: 0.001143s
Fixed Padding Time: 0.001161s
Fixed Padding Time: 0.001197s
Fixed Padding Time: 0.001000s
Fixed Padding Time: 0.001240s
Fixed Padding Time: 0.000871s
Fixed Padding Time: 0.001492s
Fixed Padding Time: 0.000919s
Fixed Padding Time: 0.001475s
Fixed Padding Time: 0.001052s
Fixed Padding Time: 0.000828s
Fixed Padding Time: 0.000800s
Fixed Padding Time: 0.001125s
Fixed Padding Time: 0.000832s
Fixed Padding Time: 0.001341s
Fixed Padding Time: 0.001148s
Fixed Padd

Random Padding Time: 0.001693s
Random Padding Time: 0.001675s
Random Padding Time: 0.001275s
Random Padding Time: 0.001417s
Random Padding Time: 0.000958s
Random Padding Time: 0.000932s
Random Padding Time: 0.000933s
Random Padding Time: 0.000923s
Random Padding Time: 0.000915s
Random Padding Time: 0.000345s
Random Padding Time: 0.000356s
Random Padding Time: 0.000417s
Random Padding Time: 0.000356s
Random Padding Time: 0.000373s
Random Padding Time: 0.000438s
Random Padding Time: 0.000360s
Random Padding Time: 0.000389s
Random Padding Time: 0.000356s
Random Padding Time: 0.000359s
Random Padding Time: 0.000352s
Random Padding Time: 0.000366s
Random Padding Time: 0.000379s
Random Padding Time: 0.000531s
Random Padding Time: 0.000552s
Random Padding Time: 0.000548s
Random Padding Time: 0.000580s
Random Padding Time: 0.001037s
Random Padding Time: 0.000633s
Random Padding Time: 0.000760s
Random Padding Time: 0.001201s
Random Padding Time: 0.001172s
Random Padding Time: 0.001198s
Random P

Fixed Padding Time: 0.001877s
Fixed Padding Time: 0.001448s
Fixed Padding Time: 0.001410s
Fixed Padding Time: 0.001260s
Fixed Padding Time: 0.000980s
Fixed Padding Time: 0.000975s
Fixed Padding Time: 0.000931s
Fixed Padding Time: 0.000919s
Fixed Padding Time: 0.000921s
Fixed Padding Time: 0.000935s
Fixed Padding Time: 0.000938s
Fixed Padding Time: 0.000921s
Fixed Padding Time: 0.000915s
Fixed Padding Time: 0.000930s
Fixed Padding Time: 0.000916s
Fixed Padding Time: 0.000917s
Fixed Padding Time: 0.000921s
Fixed Padding Time: 0.000910s
Fixed Padding Time: 0.000910s
Fixed Padding Time: 0.000985s
Fixed Padding Time: 0.000940s
Fixed Padding Time: 0.000970s
Fixed Padding Time: 0.000980s
Fixed Padding Time: 0.000918s
Fixed Padding Time: 0.001089s
Fixed Padding Time: 0.000893s
Fixed Padding Time: 0.000752s
Fixed Padding Time: 0.000732s
Fixed Padding Time: 0.001215s
Fixed Padding Time: 0.001150s
Fixed Padding Time: 0.000966s
Fixed Padding Time: 0.001135s
Fixed Padding Time: 0.001176s
Fixed Padd

Random Padding Time: 0.001428s
Random Padding Time: 0.001149s
Random Padding Time: 0.001835s
Random Padding Time: 0.001341s
Random Padding Time: 0.001376s
Random Padding Time: 0.001088s
Random Padding Time: 0.000980s
Random Padding Time: 0.001063s
Random Padding Time: 0.000925s
Random Padding Time: 0.001014s
Random Padding Time: 0.000503s
Random Padding Time: 0.000225s
Random Padding Time: 0.000360s
Random Padding Time: 0.000477s
Fixed Padding Time: 0.000354s
Fixed Padding Time: 0.000442s
Fixed Padding Time: 0.000371s
Fixed Padding Time: 0.000353s
Fixed Padding Time: 0.000386s
Fixed Padding Time: 0.000364s
Fixed Padding Time: 0.000354s
Fixed Padding Time: 0.000346s
Fixed Padding Time: 0.000351s
Fixed Padding Time: 0.000406s
Fixed Padding Time: 0.000699s
Fixed Padding Time: 0.000630s
Fixed Padding Time: 0.000677s
Fixed Padding Time: 0.000894s
Fixed Padding Time: 0.001189s
Fixed Padding Time: 0.000831s
Fixed Padding Time: 0.001074s
Fixed Padding Time: 0.001038s
Fixed Padding Time: 0.0008

...

Random Padding Time: 0.001776s
Random Padding Time: 0.001741s
Random Padding Time: 0.001045s
Random Padding Time: 0.001134s
Random Padding Time: 0.000927s
Random Padding Time: 0.001010s
Random Padding Time: 0.000792s
Random Padding Time: 0.001303s
Random Padding Time: 0.000867s
Random Padding Time: 0.001096s
Random Padding Time: 0.001140s
Random Padding Time: 0.000839s
Random Padding Time: 0.000807s
Random Padding Time: 0.001130s
Random Padding Time: 0.001077s
Random Padding Time: 0.001104s
Random Padding Time: 0.001067s
Random Padding Time: 0.000934s
Random Padding Time: 0.000987s
Random Padding Time: 0.000924s
Random Padding Time: 0.000937s
Random Padding Time: 0.001037s
Random Padding Time: 0.001169s
Random Padding Time: 0.001118s
Random Padding Time: 0.001132s
Random Padding Time: 0.000830s
Random Padding Time: 0.001132s
Random Padding Time: 0.000598s
Random Padding Time: 0.001121s
Random Padding Time: 0.001162s
Random Padding Time: 0.001139s
Random Padding Time: 0.000945s
Random P

Fixed Padding Time: 0.001658s
Fixed Padding Time: 0.001548s
Fixed Padding Time: 0.001447s
Fixed Padding Time: 0.001373s
Fixed Padding Time: 0.001213s
Fixed Padding Time: 0.001240s
Fixed Padding Time: 0.001077s
Fixed Padding Time: 0.001229s
Fixed Padding Time: 0.000797s
Fixed Padding Time: 0.000563s
Fixed Padding Time: 0.000397s
Fixed Padding Time: 0.000437s
Fixed Padding Time: 0.000347s
Fixed Padding Time: 0.000530s
Fixed Padding Time: 0.000516s
Fixed Padding Time: 0.000463s
Fixed Padding Time: 0.000575s
Fixed Padding Time: 0.000254s
Fixed Padding Time: 0.000310s
Fixed Padding Time: 0.000534s
Fixed Padding Time: 0.000621s
Fixed Padding Time: 0.000478s
Fixed Padding Time: 0.000456s
Fixed Padding Time: 0.012031s
Fixed Padding Time: 0.000625s
Fixed Padding Time: 0.000533s
Fixed Padding Time: 0.000389s
Fixed Padding Time: 0.000457s
Fixed Padding Time: 0.000953s
Fixed Padding Time: 0.000597s
Fixed Padding Time: 0.000551s
Fixed Padding Time: 0.000541s
Fixed Padding Time: 0.000533s
Fixed Padd

Random Padding Time: 0.001517s
Random Padding Time: 0.001091s
Random Padding Time: 0.001731s
Random Padding Time: 0.001281s
Random Padding Time: 0.001311s
Random Padding Time: 0.000868s
Random Padding Time: 0.000946s
Random Padding Time: 0.001110s
Random Padding Time: 0.000997s
Random Padding Time: 0.000849s
Random Padding Time: 0.000826s
Random Padding Time: 0.000825s
Random Padding Time: 0.000784s
Random Padding Time: 0.000812s
Random Padding Time: 0.000801s
Random Padding Time: 0.001153s
Random Padding Time: 0.001161s
Random Padding Time: 0.001154s
Random Padding Time: 0.000856s
Random Padding Time: 0.001380s
Random Padding Time: 0.001220s
Random Padding Time: 0.001088s
Random Padding Time: 0.001008s
Random Padding Time: 0.001176s
Random Padding Time: 0.000653s
Random Padding Time: 0.001047s
Random Padding Time: 0.001211s
Random Padding Time: 0.001164s
Random Padding Time: 0.001161s
Random Padding Time: 0.001166s
Random Padding Time: 0.001155s
Random Padding Time: 0.001154s
Random P

Fixed Padding Time: 0.000839s
Fixed Padding Time: 0.000978s
Fixed Padding Time: 0.001116s
Fixed Padding Time: 0.001883s
Fixed Padding Time: 0.001326s
Fixed Padding Time: 0.001387s
Fixed Padding Time: 0.001070s
Fixed Padding Time: 0.001096s
Fixed Padding Time: 0.001073s
Fixed Padding Time: 0.001281s
Fixed Padding Time: 0.000967s
Fixed Padding Time: 0.001133s
Fixed Padding Time: 0.001208s

Processing: b'vmLvv8hDbMuE8az5jkGYpJTuKfGVivcC7di8u6DpKRRoEf5fKW'...
Random Padding Time: 0.001230s
Random Padding Time: 0.001170s
Random Padding Time: 0.001170s
Random Padding Time: 0.001177s
Random Padding Time: 0.001173s
Random Padding Time: 0.001172s
Random Padding Time: 0.001168s
Random Padding Time: 0.001171s
Random Padding Time: 0.001223s
Random Padding Time: 0.001105s
Random Padding Time: 0.000637s
Random Padding Time: 0.000874s
Random Padding Time: 0.000919s
Random Padding Time: 0.000627s
Random Padding Time: 0.000821s
Random Padding Time: 0.000732s
Random Padding Time: 0.001027s
Random Paddin

Fixed Padding Time: 0.001705s
Fixed Padding Time: 0.001428s
Fixed Padding Time: 0.001476s
Fixed Padding Time: 0.001223s
Fixed Padding Time: 0.000772s
Fixed Padding Time: 0.000819s
Fixed Padding Time: 0.000892s
Fixed Padding Time: 0.000731s
Fixed Padding Time: 0.000725s
Fixed Padding Time: 0.000716s
Fixed Padding Time: 0.000715s
Fixed Padding Time: 0.000718s
Fixed Padding Time: 0.000730s
Fixed Padding Time: 0.000720s
Fixed Padding Time: 0.000711s
Fixed Padding Time: 0.000710s
Fixed Padding Time: 0.000715s
Fixed Padding Time: 0.000719s
Fixed Padding Time: 0.000709s
Fixed Padding Time: 0.000712s
Fixed Padding Time: 0.000720s
Fixed Padding Time: 0.000713s
Fixed Padding Time: 0.000705s
Fixed Padding Time: 0.000781s
Fixed Padding Time: 0.000814s
Fixed Padding Time: 0.000892s
Fixed Padding Time: 0.000945s
Fixed Padding Time: 0.000786s
Fixed Padding Time: 0.000752s
Fixed Padding Time: 0.000919s
Fixed Padding Time: 0.000909s
Fixed Padding Time: 0.000967s
Fixed Padding Time: 0.000910s
Fixed Padd